# Integrate own mention detection system
As mentioned in the previous section, to make this as easy to use as possible, this notebook provide a template for integrating own mention detection in the pipeline.

In [ ]:
from flask import Flask, request, Response, jsonify
import json
import spacy

app = Flask(__name__)

print("Loading Custom MD")


In [ ]:
def add_mention(mention, offset, mentions_list):
    mention_object = {
            "mention": mention,
            "offset": offset
            }
    mentions_list.append(mention_object)

**This is mostly the main part of the code you need to adapt**

change the mentions variable to something like 

    mentions = own_system.detect(text)
    
the own_system.detect method need to return a list of objects, that have the following format

    {
       "text": str,
       "offset": int
    }

In [ ]:
def detect(text):
    # CHANGE THIS LINE
    mentions = [{"text": "Napoleon", "offset": 0}, {"text": "emperor", "offset": 17}]
    return mentions

In [ ]:
def process(document):
    text = document['text']

    mentions = detect(text)

    if document['mentions'] is None:
        document['mentions'] = []

    for mention in mentions:
        mention_text = mention["text"]
        mention_offset = mention["offset"]
        add_mention(mention_text, mention_offset, document['mentions'])


The object that will be received by the web service looks as follows (example)

    {
      "componentId":"MD1",
      "document": {
        "componentId":"input",
        "mentions":[],
        "pipelineType":"NONE",
        "text":"Napoleon was the emperor of the First French Empire.", <-- the input text ------------------------------------------------
        "uri":null
      },
      "pipelineConfig":{
        "components":{
          "cg_ed":[],
          "md":[{
            "id":"MD1",
            "value":"http://host.docker.internal:5001"
          }]
        },
        "connections":[],
        "displayName":"MD + combined CG-ED",
        "endComponents":["MD1"],
        "exampleId":"md_combined_cged",
        "id":1,
        "pipelineConfigType":"complex",
        "startComponents":["MD1"]
      }
    }
    
You can test this mention detector by using the following command:

    curl http://127.0.0.1:5001/ --header "Content-Type: application/json" --request POST -d '{"document" : {"text": "Napoleon was the emperor of the First French Empire."},"pipelineConfig": "complex","componentId": "MD1"}'

In [ ]:
@app.route('/', methods=['get', 'post'])
def index():
    print("Incoming request:")
    print(request.data)
    req = json.loads(request.data)
    document = req['document']

    process(document)

    return jsonify(
            {'document' : document,
            'pipelineConfig' : req['pipelineConfig'],
            'componentId' : req['componentId']}
            )

In [ ]:
class LoggingMiddleware(object):
    def __init__(self, app):
        self._app = app

    def __call__(self, env, resp):
        errorlog = env['wsgi.errors']
        #pprint.pprint(('REQUEST', env), stream=errorlog)

        def log_response(status, headers, *args):
            #pprint.pprint(('RESPONSE', status, headers), stream=errorlog)
            return resp(status, headers, *args)

        return self._app(env, log_response)


# Run at flask startup (https://stackoverflow.com/a/55573732)
with app.app_context():
    pass

You can change the port of the web service here, after running the following code the webservice will be available.

**NOTE**: It only accepts get and post requests, trying to open the web page will result in an error with code 500.

In [ ]:
if __name__ == '__main__':
    port = 5001 # MAKE SURE YOU DON'T USE THE SAME PORT ON THE OTHER PIPELINE COMPONENTS
    print("Running app... on port: ", port)
    app.wsgi_app = LoggingMiddleware(app.wsgi_app)
    app.run(host='0.0.0.0', port=port)